In [ ]:
from dotenv import load_dotenv, find_dotenv
from operator import itemgetter
from langchain_openai import ChatOpenAI
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from langchain_community.chat_models import ChatOllama
import requests
load_dotenv(find_dotenv())

In [ ]:
def fetch_model_names(base_url):
    api_endpoint = f"{base_url}/api/tags"
    try:
        response = requests.get(api_endpoint)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        data = response.json()
        model_names = [model["name"] for model in data.get("models", [])]
        return model_names
    except requests.RequestException as e:
        return str(e)

base_url = "https://ollama.junzis.com"
model_names = fetch_model_names(base_url)
model = model_names[0]
model

In [ ]:
llm = ChatOllama(base_url="https://ollama.junzis.com", model=model)
llm.invoke("hi")

In [ ]:
import base64
from PIL import Image
import io


def encode_image_to_base64(image_path):
    # Open the image file
    with Image.open(image_path) as image:
        # Convert the image to RGB format (to ensure compatibility)
        rgb_image = image.convert("RGB")
        # Create a BytesIO object to hold the image data
        buffered = io.BytesIO()
        # Save the image to the BytesIO object as JPEG (or PNG if you prefer)
        rgb_image.save(buffered, format="JPEG")
        # Retrieve the byte data from the buffer
        image_byte_data = buffered.getvalue()
        # Encode the byte data to Base64
        base64_encoded_image = base64.b64encode(image_byte_data)
        # Convert bytes to string
        base64_string = base64_encoded_image.decode("utf-8")
    return base64_string


# Example usage
image_path = "C:\\Users\\justa\\Downloads\\4-ac-conflict.jpg"  # Replace this with your image file path
base64_code = encode_image_to_base64(image_path)
print(base64_code)

In [ ]:
import base64
from io import BytesIO
from PIL import Image

def convert_to_base64(pil_image):
    """
    Convert PIL images to Base64 encoded strings

    :param pil_image: PIL image
    :return: Base64 string
    """
    buffered = BytesIO()
    pil_image.save(buffered, format="JPEG")  # You can change the format if needed
    img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")
    return img_str

from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage
from langchain_core.output_parsers import StrOutputParser


# Function to create the prompt with text and image
def prompt_func(data):
    text = data["text"]
    image = data["image"]

    image_part = {
        "type": "image_url",
        "image_url": f"data:image/jpeg;base64,{image}",
    }

    content_parts = []

    text_part = {"type": "text", "text": text}

    content_parts.append(image_part)
    content_parts.append(text_part)

    return [HumanMessage(content=content_parts)]

# Create the chain with the prompt function, model, and output parser
chain = prompt_func | llm | StrOutputParser()

# Load and convert the image to base64
file_path = "C:\\Users\\justa\\Downloads\\4-ac-conflict-larger.jpg"
pil_image = Image.open(file_path)
image_b64 = convert_to_base64(pil_image)

# Invoke the chain with the text and image data
query_chain = chain.invoke(
    {"text": "You are seeing the image of aircrafts in conflict. Orange means aircraft in conflict and green means it is not in conflict. Each arrow has an aircraft call sign, flight level FL and speed. Based on the image decide if there is an aircraft conflict and if so make a conflict resolution plan according to ICAO standards: at least 1000ft vertical seperation or 5 NM horizontal seperation", "image": image_b64}
)

print(query_chain)

In [ ]:
ac4 = [
    -0.006516216788440943,
    -0.01843029260635376,
    0.014601406641304493,
    0.03202071040868759,
    0.02043420635163784,
    0.004988007713109255,
    -0.023199675604701042,
    0.01796543598175049,
    0.019554374739527702,
    0.00826216209679842,
    -0.02888413891196251,
    -0.002262842608615756,
    0.04244178533554077,
    -0.027164636179804802,
    0.008683063089847565,
    0.0029135539662092924,
    -0.011785514652729034,
    -0.05652494356036186,
    -0.04731535166501999,
    -0.003872769419103861,
    -0.0642157718539238,
    -0.003399479202926159,
    0.024726754054427147,
    -0.02035408839583397,
    -0.009482800029218197,
    0.011606904678046703,
    -0.003772618481889367,
    -0.05403594300150871,
    0.004505306947976351,
    -0.0005890257307328284,
    -0.02016603946685791,
    0.010622989386320114,
    -0.008631713688373566,
    0.00964097585529089,
    0.006594609469175339,
    -0.030392831191420555,
    -0.033523499965667725,
    -0.0046044946648180485,
    -0.04497433081269264,
    0.03443266078829765,
    0.020238621160387993,
    -0.026359375566244125,
    0.008110156282782555,
    0.08943431079387665,
    -0.0794864147901535,
    0.0035000171046704054,
    0.019324660301208496,
    0.019330600276589394,
    -0.029435817152261734,
    0.06039391830563545,
    0.024363107979297638,
    0.008791722357273102,
    -0.026480834931135178,
    0.019316110759973526,
    0.02862393483519554,
    0.010449225082993507,
    0.01976989023387432,
    0.022507287561893463,
    -0.020628511905670166,
    0.021487709134817123,
    0.024233626201748848,
    0.026675814762711525,
    -0.0033440033439546824,
    0.04502367973327637,
    0.029945772141218185,
    -0.027859076857566833,
    0.02027946524322033,
    0.03961329534649849,
    0.002273840829730034,
    0.026227589696645737,
    -0.008932114578783512,
    0.030120739713311195,
    0.02774946577847004,
    -0.011914830654859543,
    -0.013173029758036137,
    -0.008448692038655281,
    -0.02929285168647766,
    0.015014687553048134,
    -0.016528761014342308,
    0.04074345529079437,
    -0.025354405865073204,
    -0.051109328866004944,
    0.04346458241343498,
    0.005836524069309235,
    0.02193392999470234,
    -0.0267337579280138,
    -0.04041462019085884,
    -0.009433233179152012,
    0.03455306589603424,
    -0.009121586568653584,
    -0.040086306631565094,
    0.017963727936148643,
    -0.04181535169482231,
    0.045688215643167496,
    0.01483303215354681,
    -0.024791361764073372,
    0.08257200568914413,
    -0.023095840588212013,
    0.08288643509149551,
    0.03253880515694618,
    -0.006644208915531635,
    0.015713363885879517,
    0.019412942230701447,
    0.021355673670768738,
    0.04320063814520836,
    -0.047233253717422485,
    0.037793081253767014,
    0.00970943458378315,
    0.016156090423464775,
    0.003691521007567644,
    0.004159833304584026,
    -0.0035413631703704596,
    -0.02273583598434925,
    0.0757623016834259,
    0.01300534326583147,
    -0.0005972223589196801,
    0.047378238290548325,
    0.028848964720964432,
    0.03740870952606201,
    -0.007937975227832794,
    0.012448381632566452,
    -0.001887833815999329,
    0.025924380868673325,
    0.03178558126091957,
    0.020204631611704826,
    0.006071344017982483,
    0.026760628446936607,
    0.04294455796480179,
    -0.0399075485765934,
    0.00482747470960021,
    0.02811863273382187,
    0.019377373158931732,
    0.010550597682595253,
    0.015354547649621964,
    0.016378426924347878,
    0.03861822560429573,
    -0.025658056139945984,
    -0.014900784939527512,
    0.00895380787551403,
    -0.020050285384058952,
    -0.015162897296249866,
    0.0029277082066982985,
    0.007794880773872137,
    0.04601243510842323,
    -0.044476017355918884,
    -0.05769108980894089,
    0.052827537059783936,
    0.025444990023970604,
    0.018006764352321625,
    0.00208118068985641,
    -0.006076931953430176,
    -0.01184349786490202,
    -0.07747361809015274,
    0.006334797479212284,
    -0.029512887820601463,
    -0.02654104307293892,
    0.03215329721570015,
    -0.020628025755286217,
    0.028365252539515495,
    0.030669886618852615,
    0.02195320837199688,
    0.007270425092428923,
    0.06975264102220535,
    -0.016439275816082954,
    0.037655558437108994,
    -0.06372141093015671,
    -0.003913931082934141,
    0.00960701797157526,
    0.015042083337903023,
    0.03341135010123253,
    0.022206731140613556,
    0.03830879181623459,
    0.027261076495051384,
    -0.0015851196367293596,
    0.00730389216914773,
    -0.011870620772242546,
    0.044002171605825424,
    0.029329674318432808,
    -0.018908744677901268,
    -0.03506503626704216,
    0.014572582207620144,
    -0.019314944744110107,
    -0.01412762701511383,
    0.02971794083714485,
    0.015049665234982967,
    -0.020482923835515976,
    -0.019510647282004356,
    -0.027644164860248566,
    -0.05199817568063736,
    0.006205324549227953,
    0.0039866408333182335,
    0.012054870836436749,
    0.030801765620708466,
    -0.042925331741571426,
    -0.0318763293325901,
    -0.011449899524450302,
    0.03107651136815548,
    0.03576432913541794,
    -0.002692894311621785,
    0.03231602907180786,
    0.004749207757413387,
    0.003127038013190031,
    -0.023286180570721626,
    0.008121437393128872,
    0.009832873940467834,
    0.02438943274319172,
    -0.03132814168930054,
    0.028701234608888626,
    -0.013644286431372166,
    0.02303251624107361,
    0.024131573736667633,
    -0.04394906014204025,
    0.005304976832121611,
    -0.04381144419312477,
    0.044199373573064804,
    -0.016278356313705444,
    -0.022735314443707466,
    -0.021919885650277138,
    0.009108235128223896,
    0.046270422637462616,
    -0.13357305526733398,
    0.02357408031821251,
    -0.04208473488688469,
    -0.006670218426734209,
    0.0118448156863451,
    -0.012926674447953701,
    0.0337686613202095,
    0.0026369786355644464,
    0.026193037629127502,
    0.011916366405785084,
    -0.026402631774544716,
    -0.03413946181535721,
    0.03090376779437065,
    0.03525187075138092,
    -0.06620442122220993,
    0.05981262028217316,
    -0.06356773525476456,
    0.0347171351313591,
    0.000048466012231074274,
    0.024199476465582848,
    -0.004815577529370785,
    -0.03028908744454384,
    0.013976416550576687,
    0.005572974216192961,
    0.013499009422957897,
    -0.007544757332652807,
    0.01981949992477894,
    -0.06760706007480621,
    0.03214666619896889,
    0.00721983565017581,
    0.004259301349520683,
    0.012827360071241856,
    -0.005485268775373697,
    0.010134714655578136,
    0.010194383561611176,
    -0.016786860302090645,
    -0.0033075253013521433,
    0.027845554053783417,
    0.024653850123286247,
    -0.007697683293372393,
    -0.02416580729186535,
    0.0047896262258291245,
    0.026921691372990608,
    0.01889641210436821,
    -0.022303542122244835,
    0.08317576348781586,
    -0.019445456564426422,
    -0.018739253282546997,
    -0.024874115362763405,
    -0.0102262943983078,
    0.00937191117554903,
    0.041456662118434906,
    0.10182521492242813,
    0.0308657456189394,
    -0.05144541710615158,
    0.00039285008097067475,
    -0.002620044630020857,
    -0.015947606414556503,
    0.025380216538906097,
    -0.000879417173564434,
    0.023883668705821037,
    -0.01573953405022621,
    0.023049188777804375,
    -0.015494974330067635,
    0.004891129210591316,
    -0.028717635199427605,
    -0.020080789923667908,
    -0.0010754050454124808,
    0.0078023988753557205,
    -0.008217762224376202,
    -0.03063521347939968,
    0.03596610948443413,
    0.06835810095071793,
    -0.013717720285058022,
    -0.011645168997347355,
    -0.017200378701090813,
    -0.05423915013670921,
    0.042314767837524414,
    -0.019852299243211746,
    0.0012758226366713643,
    -0.041514765471220016,
    0.030747076496481895,
    0.01627550832927227,
    -0.017715293914079666,
    0.005516827572137117,
    -0.007731995545327663,
    0.0332392118871212,
    0.054168760776519775,
    0.03126459941267967,
    -0.008354196324944496,
    -0.013685803860425949,
    -0.03414018079638481,
    0.03855634480714798,
    0.04968157410621643,
    0.016408417373895645,
    -0.011594296433031559,
    0.0032333224080502987,
    -0.007305322680622339,
    0.025572197511792183,
    0.0013082855148240924,
    -0.028333943337202072,
    -0.03955035284161568,
    0.024784371256828308,
    0.006545647047460079,
    0.011576368473470211,
    0.013023982755839825,
    -0.08857449889183044,
    -0.02583967335522175,
    -0.012331575155258179,
    -0.02571566216647625,
    0.0029957739170640707,
    -0.009456740692257881,
    0.02823547273874283,
    0.0038499957881867886,
    -0.00040784807060845196,
    0.030228951945900917,
    0.0015105163911357522,
    -0.025964543223381042,
    -0.08826779574155807,
    -0.02397114783525467,
    -0.02384713850915432,
    0.017372526228427887,
    0.06252175569534302,
    -0.028405023738741875,
    0.02419210411608219,
    0.0022749551571905613,
    0.0007690529455430806,
    -0.010342104360461235,
    -0.020746318623423576,
    -0.09111718088388443,
    -0.01821274124085903,
    -0.010954324156045914,
    -0.007626994512975216,
    0.040698226541280746,
    0.0232760738581419,
    -0.008621240966022015,
    -0.0062913428992033005,
    0.02325316332280636,
    0.007506031077355146,
    0.035036809742450714,
    0.03216211870312691,
    -0.039189163595438004,
    0.04385160654783249,
    -0.015458676964044571,
    0.01393203530460596,
    -0.05951214209198952,
    0.0034243317786604166,
    -0.011630500666797161,
    -0.00880166795104742,
    0.014140965417027473,
    0.012956938706338406,
    -0.03842824697494507,
    -0.015999024733901024,
    -0.02444237656891346,
    -0.01277991198003292,
    0.010432553477585316,
    -0.0020088006276637316,
    0.017303474247455597,
    0.0036817279178649187,
    -0.02812386304140091,
    -0.055961575359106064,
    0.013647464103996754,
    -0.03285076469182968,
    -0.02290068008005619,
    0.009610860608518124,
    0.01842513680458069,
    -0.0044667781330645084,
    0.06403567641973495,
    -0.0253888126462698,
    -0.0161595419049263,
    0.0118223512545228,
    -0.03494780510663986,
    -0.031892385333776474,
    0.0061948648653924465,
    0.02011897601187229,
    0.045596200972795486,
    -0.014401635155081749,
    -0.001863398589193821,
    0.040122877806425095,
    0.012144286185503006,
    0.017509959638118744,
    0.019345812499523163,
    0.01217922568321228,
    0.01718871295452118,
    0.005030602216720581,
    0.008868484757840633,
    -0.04669519141316414,
    -0.021175632253289223,
    0.03197558596730232,
    0.029395058751106262,
    -0.05019674450159073,
    0.012260367162525654,
    -0.005562168080359697,
    0.04889595881104469,
    0.007523456588387489,
    0.017945483326911926,
    0.004788347519934177,
    -0.017047882080078125,
    -0.001163661596365273,
    -0.008832275867462158,
    -0.02258557826280594,
    -0.005361957009881735,
    0.019072966650128365,
    0.011462138965725899,
    -0.010796019807457924,
    0.025445204228162766,
    0.01762930490076542,
    0.006811272352933884,
    0.04494580999016762,
    0.019321469590067863,
    0.009331933222711086,
    0.0004546386771835387,
    -0.02573307789862156,
    0.008288941346108913,
    -0.03374996408820152,
    -0.05015679448843002,
    0.05165821313858032,
    0.05156625807285309,
    -0.006315759848803282,
    -0.09037855267524719,
    -0.002954769879579544,
    -0.002950637601315975,
    0.009189766831696033,
    0.010832861997187138,
    0.03991856426000595,
    0.03255946934223175,
    -0.039665430784225464,
    -0.0173114612698555,
    0.00487627275288105,
    0.017949925735592842,
    0.027768436819314957,
    0.024269768968224525,
    -0.024597030133008957,
    -0.02304784767329693,
    0.0000042683136598498095,
    0.022024080157279968,
    -0.010764854960143566,
    0.24547572433948517,
    0.06411051005125046,
    0.0007040330674499273,
    -0.01726536825299263,
    0.02126883901655674,
    0.0022473172284662724,
    0.032955244183540344,
    -0.001088903401978314,
    0.0011961569543927908,
    0.013257390819489956,
    -0.00808260589838028,
    0.013762382790446281,
    -0.02523602358996868,
    0.01660069450736046,
    -0.03857480362057686,
    0.02234557271003723,
    0.021667778491973877,
    -0.016415152698755264,
    -0.014800901524722576,
    0.03514466807246208,
    -0.07084888964891434,
    -0.03780316933989525,
    -0.010598479770123959,
    -0.011409187689423561,
    0.024792442098259926,
    0.03562088683247566,
    0.004815518856048584,
    0.028965583071112633,
    0.05451742187142372,
    0.007555617950856686,
    0.04378105700016022,
    -0.03760451078414917,
    0.0032926122657954693,
    -0.0052234940230846405,
    0.008363545872271061,
    0.055645786225795746,
    0.04839899390935898,
    -0.013476185500621796,
    0.018099864944815636,
    0.023145362734794617,
    0.0064383517019450665,
    0.001636673347093165,
    0.01639791578054428,
    0.009948649443686008,
    0.016446471214294434,
    0.041733451187610626,
    0.014126037247478962,
    0.021522972732782364,
    -0.018413754180073738,
    -0.01641123555600643,
    0.020431410521268845,
    0.032701052725315094,
    0.019899124279618263,
    0.03732719272375107,
    -0.016254587098956108,
    -0.002957005752250552,
    -0.03574182465672493,
    0.016888778656721115,
    0.059149984270334244,
    0.02321099303662777,
    -0.017769664525985718,
    0.0075787645764648914,
    0.02141634188592434,
    -0.012965627945959568,
    0.041019994765520096,
    -0.017788169905543327,
    -0.029758017510175705,
    -0.004921821411699057,
    -0.04063953459262848,
    -0.013764971867203712,
    0.014744493179023266,
    0.02877761796116829,
    0.08912529051303864,
    0.11751078814268112,
    -0.030683590099215508,
    0.004538365639746189,
    0.04044364020228386,
    -0.0007898705662228167,
    0.03414205461740494,
    -0.002736669732257724,
    -0.043817415833473206,
    -0.0009439723216928542,
    -0.008868096396327019,
    -0.020710254088044167,
    0.0010295386891812086,
    0.030826641246676445,
    -0.0011434238404035568,
    -0.026336876675486565,
    -0.018694357946515083,
    0.008314310573041439,
    -0.013409324921667576,
    0.033737875521183014,
    -0.006051225122064352,
    0.03561019524931908,
    0.007820063270628452,
    -0.02861224114894867,
    0.05732772871851921,
    -0.011206229217350483,
    0.04376868158578873,
    0.017438294366002083,
    -0.03591390699148178,
    -0.018940255045890808,
    -0.05486636608839035,
    0.02055869810283184,
    0.015512611716985703,
    0.0009787303861230612,
    -0.03189155459403992,
    -0.08673705160617828,
    -0.015465556643903255,
    0.027005407959222794,
    -0.006204192526638508,
    0.01786894164979458,
    -0.026927605271339417,
    0.022523904219269753,
    0.001057591289281845,
    -0.011163621209561825,
    0.021127620711922646,
    0.004877194762229919,
    -0.01997876912355423,
    -0.009635535068809986,
    0.006201527547091246,
    0.01154660526663065,
    0.026209084317088127,
    0.015631714835762978,
    0.0056845648214221,
    0.026479635387659073,
    0.07573447376489639,
    0.008423011749982834,
    0.04725976660847664,
    0.01943233795464039,
    0.0038735908456146717,
    -0.01769578456878662,
    0.01815783418715,
    0.006819229107350111,
    -0.004947857465595007,
    -0.0010852657724171877,
    0.015272742137312889,
    0.006509415339678526,
    0.02749151550233364,
    0.013928578235208988,
    -0.02566736377775669,
    -0.016018861904740334,
    -0.012500991113483906,
    0.008523311465978622,
    -0.009710870683193207,
    0.047669339925050735,
    -0.020565170794725418,
    0.0744495764374733,
    0.015389572829008102,
    -0.016592493280768394,
    -0.011233381927013397,
    0.013971429318189621,
    -0.003037962131202221,
    0.0343354232609272,
    -0.042685747146606445,
    0.011835921555757523,
    -0.00430246302857995,
    0.00011361363431205973,
    -0.017091356217861176,
    -0.03365413844585419,
    -0.011813318356871605,
    -0.02550160326063633,
    -0.04714687913656235,
    -0.040284108370542526,
    0.009597367607057095,
    -0.03161834925413132,
    0.03530785068869591,
    -0.01651640608906746,
    -0.00047771568642929196,
    -0.08229030668735504,
    0.038071129471063614,
    -0.005465087480843067,
    -0.0007542951498180628,
    0.0015815907390788198,
    -0.13191556930541992,
    -0.011211301200091839,
    -0.018810296431183815,
    0.030155019834637642,
    0.011752931401133537,
    0.03631393238902092,
    0.010470690205693245,
    -0.0016489940462633967,
    -0.039053115993738174,
    0.0528656430542469,
    -0.026539336889982224,
    0.005294124595820904,
    -0.014812950976192951,
    0.015186894685029984,
    0.08195654302835464,
    0.010530516505241394,
    0.019465608522295952,
    -0.04001212120056152,
    -0.010679872706532478,
    0.008056536316871643,
    0.005417949985712767,
    0.030339956283569336,
    -0.011414892971515656,
    0.028256945312023163,
    -0.005174016114324331,
    0.011334169656038284,
    0.035198092460632324,
    0.04806678369641304,
    -0.010678166523575783,
    0.02409372478723526,
    -0.012136696837842464,
    -0.05030166357755661,
    0.014709096401929855,
    -0.011309253983199596,
    -0.02561485581099987,
    -0.014450686052441597,
    -0.03701550140976906,
    -0.02820107527077198,
    -0.021063461899757385,
    -0.03999273478984833,
    0.0009912080131471157,
    -0.019902648404240608,
    -0.026581376791000366,
    -0.023958364501595497,
    0.029422614723443985,
    -0.004067074973136187,
    -0.01816590502858162,
    -0.019947178661823273,
    -0.0090751638635993,
    0.00669747032225132,
    -0.027415407821536064,
    0.01285276748239994,
    -0.029817651957273483,
    0.010039692744612694,
    -0.031955353915691376,
    0.014170144684612751,
    -0.0025203304830938578,
    0.002521806163713336,
    -0.021055519580841064,
    -0.018787512555718422,
    -0.016694922000169754,
    0.014354974962770939,
    0.011056218296289444,
    0.011605700477957726,
    0.015061198733747005,
    -0.004266917705535889,
    0.0064535317942500114,
    0.027474259957671165,
    -0.033353887498378754,
    -0.0019028432434424758,
    -0.012563329190015793,
    -0.0073630549013614655,
    0.06158220022916794,
    -0.03489026427268982,
    -0.035014353692531586,
    0.016696983948349953,
    0.009495794773101807,
    -0.0050757573917508125,
    -0.028624067083001137,
    -0.011158036068081856,
    0.034303758293390274,
    -0.0027540368027985096,
    -0.014240087941288948,
    0.015799017623066902,
    0.018141452223062515,
    0.018146377056837082,
    -0.012757540680468082,
    -0.006211713422089815,
    -0.027139268815517426,
    -0.021669676527380943,
    -0.017676059156656265,
    0.011641300283372402,
    -0.013327145017683506,
    -0.03280821442604065,
    0.011784801259636879,
    0.01703910529613495,
    0.0006820523994974792,
    -0.03372455760836601,
    0.029065752401947975,
    0.010162205435335636,
    0.016717173159122467,
    -0.07411942631006241,
    -0.008081650361418724,
    0.01693224534392357,
    -0.03071974776685238,
    0.002632336923852563,
    -0.0652192085981369,
    -0.01304803229868412,
    -0.013534653931856155,
    -0.03519375994801521,
    0.001472259871661663,
    -0.006735676433891058,
    -0.032827045768499374,
    -0.029483802616596222,
    0.03183439001441002,
    0.018169546499848366,
    0.0034266312140971422,
    0.04532860964536667,
    0.03289375826716423,
    0.025882422924041748,
    -0.0020654690451920033,
    0.015143307857215405,
    -0.01168094016611576,
    0.0290231853723526,
    -0.22151082754135132,
    0.044349104166030884,
    0.006654114928096533,
    -0.04000323265790939,
    0.06100897118449211,
    -0.038938041776418686,
    -0.025298209860920906,
    0.02618957683444023,
    0.021915143355727196,
    -0.02123785763978958,
    0.005603318568319082,
    0.013861185871064663,
    0.009476853534579277,
    0.032149121165275574,
    -0.008144836872816086,
    -0.012395570985972881,
    -0.04157821461558342,
    0.0006881504086777568,
    -0.005835399497300386,
    -0.030281512066721916,
    -0.03149634227156639,
    -0.016088929027318954,
    -0.026394639164209366,
    0.0036991736851632595,
    0.02026296965777874,
    -0.0024251609575003386,
    -0.023446084931492805,
    0.04184923321008682,
    0.026010187342762947,
    -0.033341653645038605,
    0.026557696983218193,
    0.05132216960191727,
    0.0014500065008178353,
    -0.047534190118312836,
    0.03462815284729004,
    0.06205715239048004,
    0.007686189375817776,
    -0.028773650527000427,
    0.04985698685050011,
    -0.01593683287501335,
    0.010248849168419838,
    0.02198367938399315,
    -0.023011451587080956,
    -0.011866582557559013,
    -0.008798236027359962,
    -0.024477865546941757,
    -0.004172365181148052,
    -0.010266563855111599,
    -0.000423736433731392,
    0.0004388782544992864,
    0.018318334594368935,
    -0.03295179829001427,
    0.032813046127557755,
    -0.007907385006546974,
    0.012721429578959942,
    -0.037893299013376236,
    0.0064645507372915745,
    0.02001589722931385,
    -0.01335506048053503,
    0.013750427402555943,
    0.04382682591676712,
    0.007934020832180977,
    0.023970166221261024,
    -0.0048358142375946045,
    -0.00031917347223497927,
    -0.011182531714439392,
    0.027167022228240967,
    -0.0008984401938505471,
    -0.004381794948130846,
    0.06529706716537476,
    -0.00869238842278719,
    0.02242266573011875,
    0.020934611558914185,
    0.01917581632733345,
    -0.05129290744662285,
    0.009830188006162643,
    0.015003358945250511,
    -0.036163974553346634,
    -0.031889233738183975,
    -0.003782171057537198,
    -0.052733052521944046,
    0.004205391276627779,
    -0.018093839287757874,
    -0.006541858427226543,
    -0.013842896558344364,
    0.0290011465549469,
    -0.029271738603711128,
    -0.10510766506195068,
    -0.00016743759624660015,
    -0.008386380039155483,
    -0.01712503843009472,
    0.0358523391187191,
    -0.019120536744594574,
    -0.0007233460200950503,
    0.005713365506380796,
    0.04452463611960411,
    -0.02617676742374897,
    -0.023631421849131584,
    -0.00013694095832761377,
    -0.004366590175777674,
    -0.051447540521621704,
    0.00990905798971653,
    0.007601370569318533,
    0.03311830013990402,
    0.03734850138425827,
    -0.052337680011987686,
    0.03455540910363197,
    0.012101070024073124,
    -0.012990339659154415,
    -0.023204142227768898,
    0.005221224389970303,
    0.02288789674639702,
    0.00544764706864953,
    0.047004640102386475,
    -0.04766615852713585,
    0.02304704487323761,
    0.019915729761123657,
    -0.013039845041930676,
    -0.04009298235177994,
    -0.007242563180625439,
    0.011938964948058128,
    -0.020891200751066208,
    0.047576721757650375,
    0.02624785155057907,
    -0.008281591348350048,
    -0.04039018973708153,
    0.014189621433615685,
    -0.03260042518377304,
    0.07807870954275131,
    -0.012181174010038376,
    -0.025262344628572464,
    0.03246942535042763,
    0.03130064159631729,
    0.02703830413520336,
    0.013675550930202007,
    0.0016255986411124468,
    0.02804131992161274,
    0.0011150104692205787,
    0.004301886074244976,
    -0.016176017001271248,
    0.0021522885654121637,
    -0.007348318584263325,
    0.01625613309442997,
    0.039665743708610535,
    0.0198652446269989,
    -0.038363512605428696,
    0.02139693684875965,
    -0.026571588590741158,
    -0.008000148460268974,
    0.05908282846212387,
    0.027660580351948738,
    0.025601135566830635,
    -0.007411687169224024,
    0.007506344933062792,
    0.000506049080286175,
    0.006265969946980476,
    0.0003661868686322123,
    0.0203848984092474,
    0.030262459069490433,
    -0.011038566939532757,
    0.04013611003756523,
    -0.023775601759552956,
    0.037707529962062836,
    0.014464476145803928,
    -0.06055533140897751,
    -0.00024282955564558506,
    -0.03818783164024353,
    -0.001353845582343638,
    -0.000543570495210588,
    0.015084458515048027,
    -0.0010995428310707211,
    -0.006826055236160755,
    -0.008504705503582954,
    0.024589743465185165,
    0.01978352665901184,
    0.004545789211988449,
    0.014340868219733238,
    0.011139505542814732,
    0.04819050058722496,
    -0.026537975296378136,
    -0.04893094301223755,
    -0.019213002175092697,
    0.023504992946982384,
    0.010078332386910915,
    -0.019888397306203842,
    -0.060144905000925064,
    0.0356016680598259,
    0.02189837209880352,
    -0.044275131076574326,
    -0.013038167729973793,
    0.006993792485445738,
    0.03156818449497223,
    0.01169593445956707,
    -0.04394267499446869,
    0.022844819352030754,
    -0.0028781353030353785,
    -0.01538869272917509,
    -0.031942300498485565,
    -0.025655612349510193,
    0.03905743733048439,
    0.0006020203582011163,
    -0.021223856136202812,
    0.022267227992415428,
    0.002084671752527356,
    -0.021633725613355637,
    0.0018997819861397147,
    -0.007589343469589949,
    0.03901626169681549,
    0.013235463760793209,
    -0.005986917298287153,
    0.013230808079242706,
    -0.018439846113324165,
    -0.053383585065603256,
    -0.07563799619674683,
    -0.011781496927142143,
    0.0181362796574831,
    0.017139578238129616,
    0.006087359506636858,
    -0.05621755123138428,
    -0.01825060322880745,
    0.012154027819633484,
    0.022860964760184288,
    -0.02316778525710106,
    -0.005665841978043318,
    0.015304707922041416,
    0.02953866310417652,
    -0.014767062850296497,
    -0.0017139504197984934,
    -0.007885940372943878,
    0.011075840331614017,
    0.006901893764734268,
    0.03123871237039566,
    -0.01600048504769802,
    -0.01990731619298458,
    -0.019523432478308678,
    -0.020846983417868614,
    0.045921530574560165,
    -0.05682818591594696,
    -0.04280860349535942,
    -0.01915978081524372,
    -0.03582505136728287,
    0.04986719414591789,
    -0.008589228615164757,
    -0.022207511588931084,
    0.011160345748066902,
    0.020124346017837524,
    -0.07289339601993561,
    -0.027765410020947456,
    0.022579921409487724,
    0.01957915723323822,
    0.007661581505089998,
    0.013100149109959602,
    -0.018189512193202972,
    0.03940384089946747,
    0.01911931484937668,
    -0.01794440485537052,
    -0.05693027004599571,
    0.02036362886428833,
    -0.005852350499480963,
    0.036973293870687485,
    0.03257780522108078,
    -0.057688236236572266,
    -0.05650416016578674,
    0.06497233361005783,
    -0.015932554379105568,
    -0.02316948026418686,
    0.032356563955545425,
    0.03356500715017319,
    0.04044961556792259,
]

ac2_headon = [
  -0.038061510771512985,
  0.004723926540464163,
  0.016006959602236748,
  -0.0034413421526551247,
  0.0188523530960083,
  -0.0018260382348671556,
  0.04419102892279625,
  0.019680030643939972,
  -0.02305382862687111,
  -0.016670888289809227,
  0.000054351123253582045,
  0.00676402822136879,
  0.001541307894513011,
  0.024266047403216362,
  0.026692133396863937,
  -0.03468085452914238,
  -0.020507801324129105,
  -0.030149336904287338,
  0.014115311205387115,
  -0.03749963268637657,
  -0.03737210854887962,
  -0.038420796394348145,
  0.06599795073270798,
  0.044209353625774384,
  -0.004775297362357378,
  -0.0023578451946377754,
  -0.008055323734879494,
  -0.0038417382165789604,
  -0.011407086625695229,
  -0.05118343606591225,
  0.04018929973244667,
  -0.008538315072655678,
  -0.009145183488726616,
  -0.046418413519859314,
  -0.012361840344965458,
  -0.0035659552086144686,
  -0.04393564164638519,
  0.040707286447286606,
  -0.029636602848768234,
  0.009527262300252914,
  0.007911249063909054,
  0.01577635481953621,
  0.040196534246206284,
  0.04127819836139679,
  -0.09946833550930023,
  -0.06342805922031403,
  0.004957761149853468,
  -0.05849528685212135,
  -0.0002085812739096582,
  -0.004905208013951778,
  0.004642372485250235,
  0.03712870925664902,
  0.03895367309451103,
  -0.019999651238322258,
  0.0052176970057189465,
  -0.0075972978956997395,
  0.027899591252207756,
  0.011304489336907864,
  -0.06282031536102295,
  -0.02594020962715149,
  0.027202216908335686,
  0.02293499745428562,
  -0.009312123991549015,
  0.0027725284453481436,
  0.03949274867773056,
  -0.019003909081220627,
  0.024019906297326088,
  -0.00798732042312622,
  0.005880455952137709,
  0.031828660517930984,
  -0.003039382630959153,
  0.004896130412817001,
  0.0032906036358326674,
  -0.01035222690552473,
  -0.017397837713360786,
  -0.024662267416715622,
  0.02098163403570652,
  0.007982378825545311,
  -0.002555729588493705,
  0.017962120473384857,
  -0.04207663983106613,
  -0.046139489859342575,
  0.029374781996011734,
  -0.02007679082453251,
  0.018582914024591446,
  -0.0059486692771315575,
  -0.03902833163738251,
  -0.006958840414881706,
  -0.010568187572062016,
  -0.0020486379507929087,
  -0.05229150131344795,
  0.01163686253130436,
  -0.06594691425561905,
  -0.016090217977762222,
  0.020843004807829857,
  0.020559126511216164,
  0.06056717783212662,
  -0.012343632988631725,
  0.006986138876527548,
  0.00644178269430995,
  0.03414977714419365,
  0.011804631911218166,
  0.026269543915987015,
  -0.015505895018577576,
  0.0403934046626091,
  -0.051363348960876465,
  -0.0014068727614358068,
  -0.06029365956783295,
  0.045773621648550034,
  0.023801591247320175,
  0.013938012532889843,
  0.007149668410420418,
  0.028766650706529617,
  0.047092124819755554,
  0.04350624978542328,
  -0.019688045606017113,
  0.01864016428589821,
  0.046164922416210175,
  -0.020467393100261688,
  -0.018913492560386658,
  0.007932844571769238,
  -0.021976681426167488,
  0.03697943314909935,
  -0.01737481914460659,
  0.05600014701485634,
  -0.027047444134950638,
  0.015936538577079773,
  0.024333829060196877,
  0.009925760328769684,
  -0.003985616844147444,
  0.03137807920575142,
  0.012744739651679993,
  0.03156708553433418,
  0.030432211235165596,
  -0.017333272844552994,
  0.0018475056858733296,
  -0.010209481231868267,
  -0.011433644220232964,
  -0.021352240815758705,
  -0.008041656576097012,
  -0.006560560315847397,
  0.016927063465118408,
  0.02964993752539158,
  0.07522717863321304,
  0.010015718638896942,
  0.0020087817683815956,
  0.008262483403086662,
  -0.008508440107107162,
  -0.0017253683181479573,
  0.004208666272461414,
  -0.008800715208053589,
  -0.007488388102501631,
  -0.0021325142588466406,
  -0.013293604366481304,
  -0.04197224974632263,
  -0.018638858571648598,
  -0.027689244598150253,
  -0.0615353025496006,
  -0.0026429782155901194,
  -0.00022319273557513952,
  0.005382363218814135,
  0.02022555284202099,
  0.018291831016540527,
  -0.016854457557201385,
  -0.0097655039280653,
  0.03498426452279091,
  -0.02828354761004448,
  0.015211004763841629,
  -0.007342567201703787,
  0.0053780218586325645,
  0.018534120172262192,
  0.029852885752916336,
  0.04311724379658699,
  0.018543163314461708,
  0.0008053337805904448,
  0.023719528689980507,
  0.012678627856075764,
  0.015232658013701439,
  -0.033892687410116196,
  -0.01880425214767456,
  0.03276827186346054,
  0.0005085109150968492,
  -0.0023634706158190966,
  -0.007013000547885895,
  0.007923846133053303,
  -0.01959327794611454,
  0.04373909533023834,
  0.01680147461593151,
  -0.029066916555166245,
  0.031318508088588715,
  -0.004064637236297131,
  -0.0021647121757268906,
  0.022077158093452454,
  -0.013302432373166084,
  -0.02580251358449459,
  -0.04472096264362335,
  0.01269722729921341,
  0.03454650193452835,
  0.017039868980646133,
  0.040898747742176056,
  -0.028452837839722633,
  -0.03310251608490944,
  -0.01914931833744049,
  -0.003652472048997879,
  -0.03703397512435913,
  0.03629858419299126,
  -0.0028404041659086943,
  -0.01793239824473858,
  0.02957168221473694,
  0.004732209257781506,
  -0.03713459521532059,
  -0.029112661257386208,
  -0.010423359461128712,
  -0.0062258257530629635,
  0.03396925330162048,
  -0.0012387136230245233,
  0.02478230930864811,
  0.006972945760935545,
  0.007538083475083113,
  0.05674992501735687,
  -0.04101898893713951,
  0.0138966990634799,
  -0.07485880702733994,
  -0.003875521244481206,
  0.05156787112355232,
  -0.009176768362522125,
  -0.06031780689954758,
  0.06660909950733185,
  0.04751678183674812,
  0.05373670905828476,
  -0.0380721390247345,
  0.008426844142377377,
  -0.022056156769394875,
  0.009413500316441059,
  -0.004942702129483223,
  0.02654610201716423,
  0.02958947978913784,
  -0.005534342490136623,
  -0.009276123717427254,
  -0.06565182656049728,
  0.0036356342025101185,
  0.012674015015363693,
  -0.012947514653205872,
  0.02141672745347023,
  -0.022932598367333412,
  0.010375394485890865,
  0.022483445703983307,
  -0.01723005250096321,
  0.05924128368496895,
  -0.014993663877248764,
  -0.013978693634271622,
  0.024827400222420692,
  -0.0089880945160985,
  -0.010577465407550335,
  0.004225173033773899,
  -0.043108418583869934,
  0.005673808977007866,
  0.0012179826153442264,
  0.014904088340699673,
  -0.06284821778535843,
  0.0030362808611243963,
  -0.03743458166718483,
  0.009188313968479633,
  -0.04600237309932709,
  -0.029645061120390892,
  0.04289278760552406,
  0.031466756016016006,
  0.0057598259299993515,
  -0.04372159019112587,
  0.02239173650741577,
  -0.016326704993844032,
  0.014432286843657494,
  0.05846523493528366,
  0.010587583296000957,
  -0.01711583510041237,
  -0.009023688733577728,
  0.008595442399382591,
  -0.01980685256421566,
  0.047324251383543015,
  0.017142953351140022,
  0.057503312826156616,
  0.028409844264388084,
  0.03651806712150574,
  -0.016027159988880157,
  -0.020446008071303368,
  0.0075375852175056934,
  0.02703855372965336,
  0.01107512041926384,
  -0.025387058034539223,
  0.013332468457520008,
  -0.013070655055344105,
  0.009476939216256142,
  0.01479304302483797,
  -0.02356703393161297,
  0.0000547151685168501,
  -0.010814670473337173,
  -0.015579510480165482,
  -0.008113292045891285,
  -0.003959404770284891,
  0.03506609424948692,
  -0.02196279726922512,
  0.05999092012643814,
  0.023023489862680435,
  0.019827064126729965,
  0.026923319324851036,
  -0.04699239134788513,
  0.012420875020325184,
  0.026055986061692238,
  -0.01611442118883133,
  0.0032224725000560284,
  0.016642803326249123,
  0.030617712065577507,
  0.017599929124116898,
  0.039408549666404724,
  -0.005582947749644518,
  -0.008665405213832855,
  0.010278698056936264,
  0.024454250931739807,
  -0.019189322367310524,
  -0.01273531187325716,
  0.007583852857351303,
  -0.0012329050805419683,
  -0.001954172272235155,
  0.00942247174680233,
  0.025306615978479385,
  0.017686618492007256,
  -0.07868783921003342,
  -0.02319110743701458,
  0.010713918134570122,
  -0.04171865060925484,
  -0.029786352068185806,
  -0.05346400663256645,
  -0.013639186508953571,
  -0.00937649980187416,
  -0.045740000903606415,
  0.006249214522540569,
  0.005655385088175535,
  -0.01654764451086521,
  -0.06020089238882065,
  -0.002582000335678458,
  0.04347941279411316,
  -0.0034918354358524084,
  0.047453366219997406,
  0.010460522957146168,
  -0.0147233372554183,
  0.011502444744110107,
  -0.04504599794745445,
  0.007015860639512539,
  0.015709761530160904,
  -0.08559050410985947,
  -0.03255444020032883,
  0.04595557227730751,
  0.02555857226252556,
  0.013982546515762806,
  -0.0036503190640360117,
  0.018299009650945663,
  0.03714260831475258,
  -0.0022705020383000374,
  -0.014644909650087357,
  0.06551693379878998,
  0.012960991822183132,
  -0.08267950266599655,
  0.010488873347640038,
  -0.02474193088710308,
  0.055557895451784134,
  -0.01756380684673786,
  -0.0211687870323658,
  0.0281318761408329,
  -0.01182414311915636,
  0.021924208849668503,
  0.05393997207283974,
  0.008393317461013794,
  0.00517143914476037,
  -0.005446706898510456,
  -0.012933220714330673,
  -0.00206690258346498,
  0.0019605697598308325,
  -0.021510859951376915,
  0.014081515371799469,
  0.008444616571068764,
  -0.04414070397615433,
  0.040134720504283905,
  -0.12385863810777664,
  0.0027940862346440554,
  -0.012968283146619797,
  0.0018973295809701085,
  0.017522575333714485,
  0.03228607773780823,
  0.006793419364839792,
  0.032237906008958817,
  -0.039277687668800354,
  -0.015336987562477589,
  -0.03491592779755592,
  -0.00145075807813555,
  -0.0016389187658205628,
  0.028463581576943398,
  -0.022880595177412033,
  -0.03174709528684616,
  0.06966657936573029,
  0.014939572662115097,
  0.0032570508774369955,
  0.02703029476106167,
  0.018616190180182457,
  0.019317900761961937,
  0.027247155085206032,
  0.010826162993907928,
  -0.008824451826512814,
  0.017944738268852234,
  -0.021509654819965363,
  -0.0036784708499908447,
  -0.10648027807474136,
  -0.03421838581562042,
  0.025358498096466064,
  0.00591525062918663,
  -0.0026602328289300203,
  0.025787003338336945,
  -0.011645219288766384,
  -0.0009096443536691368,
  0.02570975013077259,
  0.0027249844279140234,
  -0.027127021923661232,
  -0.005765444599092007,
  -0.04868209362030029,
  0.02338695153594017,
  -0.008766422048211098,
  0.032522693276405334,
  0.0018625428201630712,
  -0.007840435951948166,
  0.007147135213017464,
  0.03620050102472305,
  -0.04538263753056526,
  0.003576860064640641,
  0.00804667267948389,
  0.03829314559698105,
  -0.04508575052022934,
  -0.03584590181708336,
  -0.016097139567136765,
  -0.0012994656572118402,
  -0.011033971793949604,
  -0.04602199047803879,
  0.034565627574920654,
  -0.014263101853430271,
  0.02849993295967579,
  -0.009149661287665367,
  -0.019268780946731567,
  0.0009228333365172148,
  -0.0688721165060997,
  -0.051264744251966476,
  0.00978806521743536,
  0.01768430322408676,
  0.029193632304668427,
  0.008076842874288559,
  -0.0355183407664299,
  0.026916956529021263,
  0.007756042759865522,
  0.020402563735842705,
  0.010700847022235394,
  0.31563347578048706,
  -0.002858220599591732,
  0.017282158136367798,
  -0.012136045843362808,
  0.01296053547412157,
  0.01198780070990324,
  0.05736616253852844,
  0.014982860535383224,
  0.009266463108360767,
  -0.0037148999981582165,
  -0.012838820926845074,
  -0.011311803944408894,
  -0.06406842917203903,
  0.04421141743659973,
  -0.009825631976127625,
  0.06102174520492554,
  -0.041319310665130615,
  -0.012179769575595856,
  0.03220657631754875,
  0.006893447134643793,
  0.01356515847146511,
  0.011251484043896198,
  0.05503194034099579,
  -0.0037196220364421606,
  0.027413923293352127,
  -0.019484352320432663,
  -0.014235357753932476,
  0.0343308299779892,
  0.03825489059090614,
  0.004804475232958794,
  0.016373008489608765,
  -0.04853247478604317,
  0.03529491648077965,
  0.04107213765382767,
  -0.001627418794669211,
  0.010463591665029526,
  0.012524697929620743,
  0.009079876355826855,
  -0.002008874202147126,
  0.03243594989180565,
  0.022137045860290527,
  0.028465120121836662,
  0.0008484454010613263,
  -0.01801438070833683,
  0.010297794826328754,
  0.018315929919481277,
  -0.02233169414103031,
  -0.050966061651706696,
  -0.0049344561994075775,
  0.00026724115014076233,
  -0.029953762888908386,
  0.02190525084733963,
  -0.04125433415174484,
  -0.007659833878278732,
  -0.017721977084875107,
  0.01746930554509163,
  -0.0333852656185627,
  -0.011805369518697262,
  0.03133445978164673,
  -0.016470924019813538,
  0.041295457631349564,
  0.003827756503596902,
  0.007776201702654362,
  0.002084850799292326,
  0.04461465775966644,
  -0.007091368082910776,
  -0.00835941918194294,
  0.0007853673305362463,
  -0.004195950459688902,
  -0.05033116787672043,
  0.007357039488852024,
  -0.013659085147082806,
  0.0272624921053648,
  0.03337927162647247,
  0.027982335537672043,
  0.013984286226332188,
  0.04810871183872223,
  0.024870168417692184,
  0.012373757548630238,
  0.04902039095759392,
  -0.011000202037394047,
  -0.02114502526819706,
  -0.006405561696738005,
  0.03701500594615936,
  -0.03418682515621185,
  0.020117945969104767,
  0.018135014921426773,
  -0.060953520238399506,
  -0.007294717710465193,
  -0.024213731288909912,
  0.018582087010145187,
  0.023302998393774033,
  -0.011902330443263054,
  0.006235063076019287,
  0.031399235129356384,
  -0.008715063333511353,
  0.044177569448947906,
  0.03456587344408035,
  0.018025176599621773,
  0.017964445054531097,
  -0.05143949016928673,
  -0.03326456621289253,
  -0.042267221957445145,
  -0.015508774667978287,
  -0.013546556234359741,
  -0.006119188852608204,
  -0.021629096940159798,
  -0.012840617448091507,
  -0.006349231116473675,
  0.07164745777845383,
  -0.07731564342975616,
  -0.0048544141463935375,
  0.012548339553177357,
  -0.0038594305515289307,
  0.0070784371346235275,
  0.017384588718414307,
  0.0009470871300436556,
  0.025853527709841728,
  0.033382728695869446,
  -0.00441081402823329,
  -0.008444421924650669,
  0.006309041753411293,
  0.002784528536722064,
  0.008343328721821308,
  -0.013718749396502972,
  0.029198577627539635,
  0.09065679460763931,
  -0.0024006611201912165,
  -0.0511438250541687,
  -0.0070342207327485085,
  0.015256776474416256,
  -0.023749183863401413,
  0.011099005118012428,
  0.007662101648747921,
  -0.011083865538239479,
  -0.05554862320423126,
  -0.005479041486978531,
  -0.006840982940047979,
  -0.021663283929228783,
  -0.0004261907306499779,
  -0.030876094475388527,
  -0.022930514067411423,
  -0.011052004061639309,
  -0.021147912368178368,
  -0.028275616466999054,
  -0.01773049496114254,
  0.0283089280128479,
  0.042870812118053436,
  -0.013140362687408924,
  0.02591497078537941,
  -0.02674797549843788,
  0.018666712567210197,
  -0.0075849248096346855,
  -0.02306245267391205,
  -0.04406461492180824,
  -0.002220262074843049,
  -0.04119093716144562,
  -0.044872384518384933,
  0.04009505733847618,
  -0.014165920205414295,
  0.01534754317253828,
  -0.028470102697610855,
  0.021439841017127037,
  0.0035668867640197277,
  0.01200783159583807,
  -0.0006121165351942182,
  -0.007391710765659809,
  0.01803113892674446,
  -0.02174554578959942,
  -0.09805168211460114,
  0.002751152031123638,
  -0.02203545533120632,
  0.04284724220633507,
  0.0006739569362252951,
  -0.06017995625734329,
  0.0007506833062507212,
  -0.023772966116666794,
  0.01240329910069704,
  -0.00411177147179842,
  0.036353278905153275,
  0.04313298314809799,
  0.0020923451520502567,
  -0.04568888992071152,
  -0.03154793009161949,
  -0.032745618373155594,
  0.04516640678048134,
  -0.030153974890708923,
  0.012630071491003036,
  -0.025147495791316032,
  -0.037178926169872284,
  -0.005661892704665661,
  -0.03216637670993805,
  -0.05004989355802536,
  0.03900964558124542,
  0.018759023398160934,
  0.0543147549033165,
  -0.028401413932442665,
  0.027743719518184662,
  -0.012347613461315632,
  0.026243111118674278,
  0.021018531173467636,
  0.028914809226989746,
  0.030701328068971634,
  0.013613083399832249,
  -0.00900913868099451,
  0.025286825373768806,
  -0.005214004777371883,
  -0.06999126076698303,
  0.0042191906832158566,
  -0.013409774750471115,
  0.014002404175698757,
  -0.026213891804218292,
  -0.017085867002606392,
  -0.021466214209794998,
  0.01539925578981638,
  -0.03675330430269241,
  0.001619625836610794,
  -0.014565033838152885,
  0.04335181415081024,
  0.0024770565796643496,
  0.005086878314614296,
  -0.0044928560964763165,
  -0.010202289558947086,
  0.009343169629573822,
  -0.07031597942113876,
  -0.02507127821445465,
  0.03157038241624832,
  0.004557348322123289,
  0.017084114253520966,
  -0.026697685942053795,
  -0.013431830331683159,
  0.0012760072713717818,
  -0.07545474916696548,
  -0.0019235836807638407,
  -0.017319345846772194,
  0.058569613844156265,
  -0.00216521299444139,
  -0.006907187402248383,
  0.014193188399076462,
  -0.03462011367082596,
  0.015367529354989529,
  -0.018629109486937523,
  0.01774529181420803,
  -0.005160942208021879,
  -0.038979943841695786,
  0.02281235158443451,
  0.003542698919773102,
  -0.0164136104285717,
  -0.01087137870490551,
  0.02165347896516323,
  0.029445931315422058,
  -0.0016506870742887259,
  -0.013178718276321888,
  0.00292634847573936,
  0.037605296820402145,
  -0.010215451009571552,
  0.0068819052539765835,
  -0.044673144817352295,
  0.01428144983947277,
  0.02098727785050869,
  -0.016992932185530663,
  -0.018721485510468483,
  -0.015765491873025894,
  -0.011569428257644176,
  -0.01887182891368866,
  -0.0023674750700592995,
  0.009596138261258602,
  -0.02408912405371666,
  -0.013546341098845005,
  -0.011197675950825214,
  -0.014427230693399906,
  0.0014179009012877941,
  0.005789902061223984,
  0.02757664956152439,
  0.00961565226316452,
  -0.06837321072816849,
  -0.004372772760689259,
  0.0011546085588634014,
  0.010626054368913174,
  0.055133093148469925,
  -0.019187789410352707,
  -0.027703654021024704,
  0.012103135697543621,
  0.04313679412007332,
  0.019727153703570366,
  -0.02552936226129532,
  0.023334039375185966,
  -0.00011008547153323889,
  0.008771617896854877,
  0.017063984647393227,
  -0.015543042682111263,
  0.021382085978984833,
  0.017382655292749405,
  0.0027504856698215008,
  -0.003011824330314994,
  -0.0020772418938577175,
  0.02017788030207157,
  0.07125532627105713,
  -0.28203219175338745,
  0.019413966685533524,
  0.018040042370557785,
  -0.01321021281182766,
  0.018585875630378723,
  -0.002154108602553606,
  0.006788617465645075,
  0.029967261478304863,
  0.031230973079800606,
  0.01414291001856327,
  -0.017192790284752846,
  0.01959903910756111,
  0.014092320576310158,
  0.030209064483642578,
  0.005067152436822653,
  0.032848525792360306,
  -0.05092180520296097,
  -0.004552644211798906,
  0.038072578608989716,
  -0.03186196833848953,
  -0.00025460185133852065,
  0.011588937602937222,
  -0.013888724148273468,
  -0.02721616066992283,
  0.026926806196570396,
  -0.0039047347381711006,
  -0.043953508138656616,
  -0.007337664719671011,
  -0.017369026318192482,
  -0.008416270837187767,
  0.007409355603158474,
  0.04565681517124176,
  -0.041397638618946075,
  -0.037172239273786545,
  0.0189512912184,
  0.055993445217609406,
  -0.0014308578101918101,
  -0.042980436235666275,
  0.04264454171061516,
  0.04892421513795853,
  -0.027756577357649803,
  0.05692289024591446,
  0.00752387847751379,
  0.0008381889201700687,
  0.028336482122540474,
  0.015315383672714233,
  0.0203548576682806,
  0.025590989738702774,
  0.005064510740339756,
  0.005136970896273851,
  0.01736794039607048,
  -0.01833421364426613,
  -0.010221507400274277,
  0.011682067066431046,
  0.029978124424815178,
  0.05720701068639755,
  -0.013915462419390678,
  0.02381068468093872,
  -0.0016352448146790266,
  0.019959479570388794,
  0.014292747713625431,
  0.028826240450143814,
  0.02996215596795082,
  0.02274976298213005,
  -0.009760262444615364,
  -0.051458101719617844,
  -0.02400573343038559,
  -0.00016227111336775124,
  0.03282107785344124,
  -0.03909086063504219,
  -0.03426053375005722,
  0.000056460932682966813,
  -0.005224667489528656,
  0.04984910413622856,
  -0.052974291145801544,
  0.01573742926120758,
  -0.0065805925987660885,
  -0.01667262241244316,
  0.018194017931818962,
  0.013995868153870106,
  0.01027638465166092,
  0.0130394222214818,
  -0.007495129015296698,
  0.03002762421965599,
  0.008349052630364895,
  0.009886166080832481,
  -0.02398647926747799,
  -0.009623842313885689,
  0.01354257296770811,
  0.005334902089089155,
  -0.0039113350212574005,
  0.020583946257829666,
  0.0059763966128230095,
  -0.017336823046207428,
  0.01089899055659771,
  0.021955177187919617,
  -0.013555984012782574,
  -0.01999545842409134,
  -0.012651205062866211,
  0.02624678425490856,
  -0.0027385849971324205,
  0.010082910768687725,
  0.026427991688251495,
  -0.017216665670275688,
  0.00009005882020574063,
  -0.020759571343660355,
  0.0019324561581015587,
  0.02296881377696991,
  -0.01518174447119236,
  -0.013006621971726418,
  -0.009931407868862152,
  0.07698284089565277,
  -0.003457373008131981,
  0.008107367902994156,
  -0.02963137999176979,
  0.015574634075164795,
  0.041970930993556976,
  -0.0055049811489880085,
  -0.01758101023733616,
  0.015068131498992443,
  -0.011194480583071709,
  -0.02517516538500786,
  -0.020432617515325546,
  0.018297716975212097,
  -0.0013555362820625305,
  -0.062051042914390564,
  0.016928430646657944,
  0.021842507645487785,
  -0.005325382575392723,
  -0.007815116085112095,
  0.008962983265519142,
  -0.04120459407567978,
  0.0534079372882843,
  0.00570721784606576,
  0.007373255677521229,
  0.04778159409761429,
  0.05136207118630409,
  -0.019769953563809395,
  0.019962718710303307,
  -0.04448108375072479,
  -0.0017403310630470514,
  -0.013545730151236057,
  0.04891449958086014,
  0.022479459643363953,
  0.06804008781909943,
  0.002951118629425764,
  0.01140908058732748,
  -0.012851838022470474,
  -0.021935749799013138,
  0.012454599142074585,
  0.014741849154233932,
  -0.025969799607992172,
  -0.008922852575778961,
  -0.014882908202707767,
  0.013322018086910248,
  -0.014347050338983536,
  -0.029057566076517105,
  0.05178272724151611,
  -0.029303057119250298,
  0.003800861770287156,
  0.025768449530005455,
  0.0030251522548496723,
  0.04377717152237892,
  0.028614314272999763,
  -0.01036143023520708,
  -0.022960137575864792,
  -0.016079984605312347,
  -0.032026637345552444,
  -0.0025263652205467224,
  0.004385084845125675,
  -0.013098149560391903,
  -0.04926471784710884,
  0.0212249755859375,
  0.005081630777567625,
  0.027866646647453308,
  -0.004580707289278507,
  -0.010931415483355522,
  0.012039268389344215,
  0.02028081752359867,
  0.0063194031827151775,
  -0.012809489853680134,
  -0.026796292513608932,
  0.029643062502145767,
  0.008321598172187805,
  -0.04603568837046623,
  -0.003616625675931573,
  -0.0058867232874035835,
  -0.04924774542450905,
  -0.029149923473596573,
  0.005480894818902016,
  0.02611599490046501,
  -0.002325337380170822,
  0.01381662581115961,
  -0.05213959142565727,
  0.002603737637400627,
  0.01510861050337553,
  0.005197829566895962,
  -0.003999215085059404,
  0.025767169892787933,
  -0.028235500678420067,
  0.0009571017581038177,
  -0.016179606318473816,
  0.06813276559114456,
  -0.018725251778960228,
  0.0402367040514946,
  -0.023220250383019447,
  -0.0013432760024443269,
  0.031584326177835464,
  0.016891196370124817,
  0.018963459879159927,
  -0.032453153282403946,
  0.023880435153841972,
  -0.0878005176782608,
  -0.02552560158073902,
  0.011180846951901913,
  -0.003392075188457966,
  -0.017719753086566925,
  0.004143652506172657,
  -0.059113483875989914,
  -0.022321954369544983,
  0.012047299183905125,
  -0.012825548648834229,
  -0.05512824282050133,
  0.02026710845530033,
  0.01147227268666029,
  0.03053261898458004,
  -0.004008111078292131,
  -0.004231416154652834,
  -0.012933962047100067,
  -0.040716301649808884,
  0.020484229549765587,
  0.06216736510396004,
  -0.022020189091563225,
  -0.00013157395005691797,
  -0.016271047294139862,
  -0.010803040117025375,
  0.0015637268079444766,
  -0.015144742093980312,
  -0.032524898648262024,
  -0.014856516383588314,
  -0.02871457114815712,
  0.02240244299173355,
  -0.060198310762643814,
  0.02696812152862549,
  0.023321084678173065,
  0.028025662526488304,
  0.029323410242795944,
  0.02519312873482704,
  0.01447679940611124,
  0.005274596624076366,
  -0.03153034672141075,
  0.008297099731862545,
  -0.021832454949617386,
  0.013695829547941685,
  0.05462051182985306,
  0.009571480564773083,
  -0.029197748750448227,
  -0.02031494490802288,
  0.03964030742645264,
  0.029848499223589897,
  -0.030807100236415863,
  -0.022830989211797714,
  -0.04824938252568245,
  -0.0053284247405827045,
  0.0019195284694433212,
  0.008846398442983627,
  -0.000023404229068546556,
  0.03756354749202728,
  0.04036875069141388
]


ac2_headon_2 = [
    -0.009398630820214748,
    -0.027384430170059204,
    -0.008307765237987041,
    0.058101095259189606,
    0.006406252738088369,
    0.0003333250351715833,
    -0.02017810195684433,
    0.05540592223405838,
    -0.02145334519445896,
    -0.005254141055047512,
    0.005398629233241081,
    0.04101526737213135,
    0.02593681775033474,
    0.013968756422400475,
    -0.017619138583540916,
    -0.01741367019712925,
    0.007085203193128109,
    -0.05151591822504997,
    -0.008070837706327438,
    -0.05695036053657532,
    -0.03040107898414135,
    -0.037663377821445465,
    0.007214634213596582,
    0.055985499173402786,
    0.005964531563222408,
    -0.027489038184285164,
    -0.0028556669130921364,
    -0.03903871402144432,
    -0.0013734610984101892,
    0.0193594042211771,
    0.042393554002046585,
    -0.005537146702408791,
    0.03237411752343178,
    0.020608222112059593,
    -0.03532612323760986,
    -0.039379484951496124,
    -0.006586514413356781,
    -0.053538236767053604,
    -0.02227747067809105,
    -0.010660411790013313,
    0.03471960127353668,
    -0.029724102467298508,
    0.0071570174768567085,
    0.001761959632858634,
    -0.09683016687631607,
    -0.014895224943757057,
    0.02489936351776123,
    -0.034868352115154266,
    -0.000010362620741943829,
    0.008757030591368675,
    0.002222102601081133,
    0.004934614058583975,
    0.039273206144571304,
    -0.0431474894285202,
    -0.010548345744609833,
    0.023342255502939224,
    0.017479771748185158,
    0.002093058545142412,
    -0.009597132913768291,
    -0.003179426770657301,
    0.010389438830316067,
    -0.002298226347193122,
    -0.02279520221054554,
    -0.033013150095939636,
    0.014654491096735,
    -0.05468350276350975,
    0.03539781644940376,
    0.01854236051440239,
    0.02308006025850773,
    0.032030269503593445,
    0.012709531001746655,
    0.01181641686707735,
    -0.02133217826485634,
    0.018160302191972733,
    -0.0337408147752285,
    -0.00758172944188118,
    -0.019955769181251526,
    -0.008732250891625881,
    -0.009799093939363956,
    0.03160585090517998,
    -0.0383109413087368,
    -0.06813444942235947,
    -0.0034924233332276344,
    -0.017964143306016922,
    0.005205670837312937,
    -0.016050759702920914,
    -0.02803667262196541,
    -0.015872661024332047,
    0.024749601259827614,
    -0.0020793473813682795,
    -0.05216752365231514,
    0.05170402675867081,
    -0.06802187860012054,
    0.02058466151356697,
    0.04991092532873154,
    -0.007844166830182076,
    0.04477819800376892,
    0.014420785009860992,
    0.034021422266960144,
    0.024671562016010284,
    0.032144997268915176,
    0.013693996705114841,
    0.03264564648270607,
    -0.007582165766507387,
    0.05909249559044838,
    -0.11084310710430145,
    0.045842841267585754,
    -0.015403514727950096,
    0.0365002416074276,
    0.031399406492710114,
    -0.023774709552526474,
    -0.0068868123926222324,
    -0.023093629628419876,
    0.05275917425751686,
    0.03521241620182991,
    0.003449509385973215,
    0.053270816802978516,
    -0.009540673345327377,
    -0.026982823386788368,
    -0.008624776266515255,
    0.008959782309830189,
    0.024798057973384857,
    0.02334454469382763,
    -0.0005243810010142624,
    0.0448124036192894,
    0.009887645952403545,
    0.05570092052221298,
    0.03988434001803398,
    -0.016039539128541946,
    -0.013289493508636951,
    0.010154873132705688,
    0.02235467918217182,
    0.05349719151854515,
    0.0009925136109814048,
    0.0052862917073071,
    0.02662850357592106,
    -0.01056627742946148,
    0.006134117022156715,
    0.0028269183821976185,
    -0.0008591403602622449,
    0.00526281725615263,
    0.020391471683979034,
    0.010657011531293392,
    0.005448616109788418,
    -0.015066041611135006,
    -0.019835904240608215,
    0.04932050779461861,
    0.02167302370071411,
    0.007970377802848816,
    0.020437927916646004,
    0.01409101765602827,
    -0.004852492827922106,
    0.013039065524935722,
    0.028495578095316887,
    -0.025302903726696968,
    0.009267043322324753,
    0.0037022463511675596,
    -0.008532426320016384,
    -0.0016447228845208883,
    0.011091632768511772,
    0.03382390737533569,
    0.00010586313146632165,
    0.03613809496164322,
    0.011068273335695267,
    0.0003609555715229362,
    0.004091987386345863,
    0.012837293557822704,
    0.003103594295680523,
    -0.0239592045545578,
    0.0043602753430604935,
    -0.002910859417170286,
    0.04375525563955307,
    -0.0038998560048639774,
    0.008525026962161064,
    -0.055112168192863464,
    -0.007312188856303692,
    0.017966313287615776,
    0.03829692304134369,
    -0.02699170634150505,
    -0.006898592691868544,
    0.0020124095026403666,
    0.02712998166680336,
    -0.010090074501931667,
    -0.0027800339739769697,
    -0.017549334093928337,
    -0.048894718289375305,
    -0.03033054806292057,
    0.017043793573975563,
    -0.04584493860602379,
    0.010964765213429928,
    0.03517637774348259,
    0.0015021516010165215,
    0.046884823590517044,
    -0.028668399900197983,
    0.019354304298758507,
    -0.00019979855278506875,
    0.04486668109893799,
    0.04204756021499634,
    -0.015218358486890793,
    0.024410143494606018,
    -0.027047613635659218,
    -0.05079437419772148,
    -0.03246012702584267,
    -0.04459806904196739,
    -0.03509403392672539,
    0.024317245930433273,
    -0.015305607579648495,
    -0.05772946774959564,
    -0.00003902884418494068,
    0.03267843276262283,
    -0.02411613240838051,
    -0.0016092509031295776,
    -0.0030243860092014074,
    0.006512599065899849,
    0.00809508003294468,
    -0.030537428334355354,
    -0.016563577577471733,
    -0.017599748447537422,
    0.013829641975462437,
    0.05535145476460457,
    -0.113504558801651,
    0.0008173265377990901,
    -0.03183634206652641,
    0.00804362166672945,
    0.013852792792022228,
    -0.0378723070025444,
    -0.015375638380646706,
    0.04096391797065735,
    -0.025467410683631897,
    0.00560679892078042,
    -0.05804561823606491,
    -0.025891980156302452,
    -0.0053781382739543915,
    -0.015078998170793056,
    -0.05915607139468193,
    0.015923701226711273,
    0.01725836470723152,
    -0.024761317297816277,
    0.03127254173159599,
    -0.05257483199238777,
    0.022779684513807297,
    -0.04036886245012283,
    0.0004203334974590689,
    0.03342854231595993,
    -0.003546710591763258,
    -0.0072129862383008,
    0.021909354254603386,
    -0.037296149879693985,
    -0.0019648591987788677,
    0.0006232971791177988,
    0.017750930041074753,
    0.0030626438092440367,
    -0.03714059293270111,
    0.007494254037737846,
    0.008940077386796474,
    -0.019953235983848572,
    -0.04243577644228935,
    -0.008166427724063396,
    0.023067686706781387,
    -0.026673149317502975,
    0.014759689569473267,
    -0.004104022402316332,
    0.004717297852039337,
    -0.016728822141885757,
    -0.0083731384947896,
    0.051651064306497574,
    0.020293613895773888,
    0.013825098983943462,
    -0.007079815026372671,
    0.012802420184016228,
    -0.009208695031702518,
    0.018145481124520302,
    0.11561523377895355,
    0.012388747185468674,
    -0.03187961131334305,
    -0.029434839263558388,
    -0.03558298945426941,
    0.01569179818034172,
    0.05858665704727173,
    0.0031026832293719053,
    0.04892721399664879,
    -0.009911156259477139,
    -0.02621488645672798,
    -0.0015228583943098783,
    -0.034648578613996506,
    -0.018526380881667137,
    0.008237162604928017,
    -0.011738874949514866,
    -0.005940910428762436,
    0.048391249030828476,
    -0.007043876219540834,
    0.03599017485976219,
    0.02991955354809761,
    -0.005250074900686741,
    -0.03549623489379883,
    -0.005584390368312597,
    -0.05269201099872589,
    -0.000799026049207896,
    -0.03793824836611748,
    0.035275690257549286,
    -0.015175639651715755,
    0.08925066888332367,
    0.022070759907364845,
    -0.047805383801460266,
    0.011471100151538849,
    0.02260805107653141,
    -0.027216140180826187,
    0.006904949434101582,
    -0.009908409789204597,
    -0.03827241063117981,
    0.018634825944900513,
    -0.007542422506958246,
    0.059018298983573914,
    0.035049114376306534,
    0.03579152375459671,
    -0.04193351790308952,
    -0.011190805584192276,
    -0.005186466034501791,
    0.022022748365998268,
    0.012889293022453785,
    -0.03174398094415665,
    -0.03400394693017006,
    -0.004550325218588114,
    -0.008379712700843811,
    0.0285786185413599,
    -0.02579476684331894,
    -0.11202165484428406,
    -0.018336446955800056,
    0.007863461971282959,
    -0.05073774978518486,
    -0.03078477457165718,
    -0.015458068810403347,
    -0.012008285149931908,
    -0.013927321881055832,
    0.029246220365166664,
    0.0015701756346970797,
    0.03991090878844261,
    -0.021193306893110275,
    -0.0491027794778347,
    -0.024313554167747498,
    -0.009851067326962948,
    0.004600103944540024,
    0.07909217476844788,
    0.01584687829017639,
    -0.004919336177408695,
    0.03419903293251991,
    -0.026384789496660233,
    -0.022347763180732727,
    0.0019163907272741199,
    -0.07221435755491257,
    -0.01630181260406971,
    0.012017120607197285,
    0.026781165972352028,
    0.02675170823931694,
    0.008518188260495663,
    -0.028579089790582657,
    0.019366266205906868,
    -0.004720548167824745,
    -0.013701376505196095,
    0.018884744495153427,
    0.005252585280686617,
    -0.06643325835466385,
    0.05723808705806732,
    -0.005471311509609222,
    0.04213378578424454,
    -0.07396313548088074,
    0.001011790707707405,
    0.03394413739442825,
    0.007537685800343752,
    0.03011990524828434,
    0.00921779777854681,
    -0.03381791338324547,
    -0.024364683777093887,
    -0.03270994499325752,
    -0.005908648949116468,
    0.03548607975244522,
    -0.0015650277491658926,
    0.00755678815767169,
    0.011286096647381783,
    0.03738889843225479,
    -0.04634815454483032,
    0.08156943321228027,
    -0.05744456127285957,
    -0.0503673329949379,
    -0.009970484301447868,
    0.0025161132216453552,
    -0.008426067419350147,
    0.03923105075955391,
    0.015837088227272034,
    0.033461447805166245,
    -0.0014072388876229525,
    -0.007516568526625633,
    -0.06542246788740158,
    0.010099205188453197,
    0.026079636067152023,
    0.03366280347108841,
    0.02710764668881893,
    -0.06636631488800049,
    0.04598141461610794,
    0.01820240169763565,
    0.018785331398248672,
    0.023234127089381218,
    0.0571521557867527,
    0.013944791629910469,
    0.01348842866718769,
    -0.0030673325527459383,
    -0.05853237211704254,
    0.00582874333485961,
    0.0026164897717535496,
    0.010272381827235222,
    -0.054366614669561386,
    -0.0278342142701149,
    0.022729290649294853,
    -0.000129033433040604,
    0.024752818048000336,
    0.0028323112055659294,
    0.0029760017059743404,
    -0.03294822573661804,
    0.02949855662882328,
    0.0003158332547172904,
    -0.01007937267422676,
    0.010848911479115486,
    0.00021386142179835588,
    0.012112250551581383,
    -0.006352992728352547,
    0.02882610633969307,
    0.0047080786898732185,
    0.010376853868365288,
    0.0346529483795166,
    0.015641072764992714,
    -0.00562862167134881,
    0.004929959774017334,
    -0.024678748100996017,
    0.05258338153362274,
    -0.04275180399417877,
    -0.016985172405838966,
    0.023648016154766083,
    0.013223577290773392,
    0.006805513519793749,
    -0.04988785833120346,
    -0.0009637288749217987,
    -0.05280786380171776,
    0.017669009044766426,
    0.04631572961807251,
    -0.028672771528363228,
    0.028538454324007034,
    -0.045263972133398056,
    -0.016025453805923462,
    0.009343019686639309,
    0.032620806246995926,
    0.08052408695220947,
    0.021647052839398384,
    -0.02317928709089756,
    0.006559594068676233,
    -0.011158562265336514,
    0.003968796227127314,
    -0.012329650111496449,
    0.19653837382793427,
    0.030989350751042366,
    -0.03842860832810402,
    -0.009946893900632858,
    0.007819440215826035,
    -0.006334016099572182,
    0.02532663755118847,
    0.0014790169661864638,
    0.010546955280005932,
    -0.006092752330005169,
    0.001803188701160252,
    0.024501515552401543,
    -0.002677804557606578,
    -0.013161896727979183,
    -0.00944176223129034,
    0.026723280549049377,
    -0.04547005891799927,
    0.017796071246266365,
    -0.014909258112311363,
    0.0015034015523269773,
    -0.019943464547395706,
    -0.02516436204314232,
    0.04696914181113243,
    -0.0012347418814897537,
    0.022794179618358612,
    0.0609184093773365,
    0.006334575824439526,
    0.06269728392362595,
    0.022833913564682007,
    -0.03234075382351875,
    0.017860479652881622,
    0.009662944823503494,
    0.023507490754127502,
    0.004719150718301535,
    0.00576202804222703,
    0.033725906163454056,
    0.01510755717754364,
    0.015320900827646255,
    0.016614766791462898,
    0.015916932374238968,
    0.0009922273457050323,
    0.02985287643969059,
    0.010295763611793518,
    -0.01823491044342518,
    -0.019845644012093544,
    0.0544607900083065,
    -0.012013035826385021,
    -0.02155691385269165,
    -0.0600481741130352,
    0.003522006096318364,
    -0.01766088791191578,
    0.011466491036117077,
    -0.0508999302983284,
    0.00750199519097805,
    0.0037019921001046896,
    -0.008274615742266178,
    -0.04737277328968048,
    0.01208917610347271,
    0.027985064312815666,
    -0.011725868098437786,
    -0.002681329846382141,
    0.001795845222659409,
    0.03901846706867218,
    0.020225731655955315,
    0.06428232789039612,
    0.0008945866138674319,
    -0.06525193154811859,
    -0.00046035810373723507,
    -0.048833366483449936,
    -0.024413753300905228,
    0.01892063021659851,
    0.007138112094253302,
    0.027914846315979958,
    0.09427160769701004,
    -0.024058913812041283,
    0.028237689286470413,
    0.017956692725419998,
    0.01918485201895237,
    0.04504784941673279,
    -0.0015088377986103296,
    -0.026164622977375984,
    -0.01199047826230526,
    -0.00443643145263195,
    0.024801425635814667,
    -0.03343295305967331,
    0.035488929599523544,
    0.0650705024600029,
    -0.03633534908294678,
    -0.01703820191323757,
    0.009359311312437057,
    -0.02914384938776493,
    0.03267589583992958,
    -0.041085220873355865,
    0.03593968600034714,
    0.03102804720401764,
    -0.03768803924322128,
    0.0032575230579823256,
    0.006597048137336969,
    0.031225042417645454,
    -0.005788455251604319,
    -0.03906721994280815,
    -0.004781993571668863,
    -0.09889081865549088,
    -0.013136273249983788,
    -0.046823687851428986,
    0.009144396521151066,
    0.0018146953079849482,
    -0.06635747849941254,
    -0.018854862079024315,
    0.053743064403533936,
    -0.03816836327314377,
    0.016315309330821037,
    0.008008934557437897,
    0.06573808938264847,
    0.010428329929709435,
    -0.03471960499882698,
    0.01914088986814022,
    0.02138224057853222,
    -0.018015297129750252,
    0.03238150104880333,
    0.0065096402540802956,
    0.045431897044181824,
    0.042651161551475525,
    0.018260419368743896,
    -0.01779705286026001,
    0.008770737797021866,
    0.0953519269824028,
    0.004181536380201578,
    -0.00947947334498167,
    0.027045300230383873,
    -0.004002789035439491,
    -0.006508361082524061,
    -0.04253369942307472,
    0.02234160713851452,
    0.02407865785062313,
    -0.0030839452520012856,
    -0.005015235859900713,
    0.01867823116481304,
    0.048460956662893295,
    -0.021973801776766777,
    -0.012940951623022556,
    -0.006020034663379192,
    -0.029245777055621147,
    0.03911993280053139,
    0.004714194685220718,
    -0.019111426547169685,
    0.026762699708342552,
    0.053225550800561905,
    -0.011618359945714474,
    -0.010232707485556602,
    -0.026698393747210503,
    0.0484042726457119,
    -0.009317087009549141,
    0.0014961573760956526,
    -0.04669183865189552,
    0.009001784957945347,
    -0.045641444623470306,
    0.008498438633978367,
    -0.008959019556641579,
    -0.03437008708715439,
    -0.023156652227044106,
    -0.041793398559093475,
    -0.008656144142150879,
    0.0018774797208607197,
    -0.024846583604812622,
    0.01599651202559471,
    -0.011023949831724167,
    0.031907178461551666,
    0.008846818469464779,
    -0.11643467843532562,
    0.04083085060119629,
    -0.0004782657779287547,
    -0.002265571616590023,
    -0.036975450813770294,
    -0.14105817675590515,
    -0.014360149390995502,
    0.005850557237863541,
    -0.007001157850027084,
    0.02108203060925007,
    0.0018442465225234628,
    -0.012747865170240402,
    -0.01750321313738823,
    -0.06591413170099258,
    0.0006730455788783729,
    -0.002744670258834958,
    0.02016761526465416,
    -0.013272016309201717,
    -0.02668873965740204,
    0.03618057444691658,
    0.009767549112439156,
    0.03579705208539963,
    0.01272848155349493,
    -0.03184516727924347,
    -0.010633268393576145,
    -0.015337035059928894,
    0.007683806587010622,
    -0.019901426509022713,
    0.0017017193604260683,
    0.027314897626638412,
    0.033908188343048096,
    0.06746196001768112,
    0.0039606401696801186,
    0.019580597057938576,
    0.0038882072549313307,
    0.02791331708431244,
    -0.006607865449041128,
    -0.0002395785559201613,
    -0.02078259363770485,
    0.018825825303792953,
    -0.01641693525016308,
    -0.011090180836617947,
    -0.03647814318537712,
    0.027304409071803093,
    -0.022967323660850525,
    -0.034383662045001984,
    -0.03378253057599068,
    0.031226707622408867,
    -0.01768660545349121,
    0.046600695699453354,
    0.00021454189845826477,
    -0.03894907608628273,
    0.01631331443786621,
    0.01778949797153473,
    0.008278845809400082,
    -0.060194164514541626,
    -0.01819063350558281,
    -0.003465185407549143,
    0.008714277297258377,
    -0.015508120879530907,
    0.009000254794955254,
    -0.007487453054636717,
    -0.025672828778624535,
    -0.05378243699669838,
    -0.04401528462767601,
    -0.004932863172143698,
    0.047263242304325104,
    0.0203420240432024,
    0.027195783331990242,
    0.0022072556894272566,
    -0.02507484145462513,
    0.009938104078173637,
    -0.01679339073598385,
    -0.04663950577378273,
    0.019213877618312836,
    -0.014416609890758991,
    0.0027733936440199614,
    0.027655290439724922,
    0.040954574942588806,
    -0.014535670168697834,
    0.002112041926011443,
    0.017560983076691628,
    0.035728782415390015,
    0.002497983630746603,
    -0.018723489716649055,
    0.024491045624017715,
    -0.024333341047167778,
    0.01153270062059164,
    -0.015189506113529205,
    0.011491561308503151,
    -0.001938462257385254,
    -0.017876820638775826,
    -0.003923548385500908,
    -0.017164058983325958,
    -0.034393686801195145,
    -0.005103704985231161,
    0.004274232778698206,
    0.031685493886470795,
    -0.046351172029972076,
    -0.019869668409228325,
    -0.006729676853865385,
    0.0038678317796438932,
    -0.042329899966716766,
    0.03122933954000473,
    0.005373246502131224,
    0.014785530045628548,
    -0.03164476528763771,
    -0.008422872982919216,
    0.0014621077571064234,
    -0.0107475146651268,
    0.055186036974191666,
    -0.046979423612356186,
    -0.013685560785233974,
    0.0007295120740309358,
    0.008425282314419746,
    -0.020971227437257767,
    0.013082527555525303,
    0.005021981429308653,
    -0.016078639775514603,
    0.0000861290463944897,
    0.019127048552036285,
    -0.026446303352713585,
    0.02210818976163864,
    0.040742579847574234,
    0.0011992552317678928,
    -0.02046281099319458,
    0.017103467136621475,
    -0.012673009186983109,
    0.023691119626164436,
    -0.18114803731441498,
    0.03628677502274513,
    0.025262700393795967,
    -0.007954832166433334,
    0.0865182876586914,
    -0.036383699625730515,
    -0.04507837072014809,
    0.005557627882808447,
    0.05557504668831825,
    -0.02325631119310856,
    0.0109336469322443,
    -0.021168630570173264,
    -0.04801377281546593,
    0.01542617380619049,
    -0.0020139708649367094,
    0.010300295427441597,
    -0.00916488841176033,
    -0.011551576666533947,
    -0.009753286838531494,
    -0.031048348173499107,
    0.01398442778736353,
    0.02025420218706131,
    -0.01966843381524086,
    -0.014683890156447887,
    0.0042032659985125065,
    0.052762918174266815,
    -0.038350462913513184,
    0.03153387829661369,
    0.0006881606532260776,
    -0.0519222617149353,
    0.0332166887819767,
    0.026011575013399124,
    -0.046006519347429276,
    -0.05915653333067894,
    -0.0044042677618563175,
    0.06923237442970276,
    -0.007382980082184076,
    -0.04166616126894951,
    -0.000992550398223102,
    0.010881750844419003,
    0.017744725570082664,
    0.0411432608962059,
    0.011651908978819847,
    -0.024703824892640114,
    0.03346897289156914,
    0.015076418407261372,
    0.009857794269919395,
    0.0209190770983696,
    0.0003198389895260334,
    -0.032368019223213196,
    0.01375146210193634,
    -0.017635030671954155,
    -0.026595745235681534,
    0.03958098217844963,
    0.005263613071292639,
    -0.021316125988960266,
    -0.011454376392066479,
    0.0028114954475313425,
    -0.0251148771494627,
    0.021084662526845932,
    0.006998161319643259,
    0.015854422003030777,
    0.054969657212495804,
    0.007307971362024546,
    -0.002085163025185466,
    -0.05061287805438042,
    -0.008037184365093708,
    -0.009870713576674461,
    0.021381618455052376,
    -0.0035618660040199757,
    0.019245095551013947,
    -0.017398206517100334,
    -0.003713576588779688,
    0.027551064267754555,
    -0.06437951326370239,
    0.00585874542593956,
    0.025231555104255676,
    -0.051965899765491486,
    0.001169791561551392,
    0.047660525888204575,
    -0.0021161455661058426,
    0.04415206238627434,
    -0.007280389312654734,
    -0.015796249732375145,
    -0.036064714193344116,
    -0.019533470273017883,
    -0.0015314074698835611,
    -0.03315277770161629,
    -0.005828936118632555,
    -0.012978900223970413,
    -0.009477480314671993,
    -0.0029999115504324436,
    0.049314793199300766,
    -0.030226746574044228,
    0.03047359548509121,
    0.08420171588659286,
    -0.005409396719187498,
    -0.03742390498518944,
    0.02400248870253563,
    0.003266681917011738,
    -0.010831736028194427,
    0.023741308599710464,
    0.048993781208992004,
    0.04244169592857361,
    -0.01576157659292221,
    -0.02545304223895073,
    0.020677614957094193,
    -0.013430092483758926,
    0.003305715275928378,
    0.0030093039385974407,
    0.005376220215111971,
    0.049788814038038254,
    0.030644189566373825,
    0.01094409916549921,
    -0.04831497371196747,
    0.02821405790746212,
    -0.005581913515925407,
    0.00862613320350647,
    -0.0009731915779411793,
    0.03849681094288826,
    0.011688192375004292,
    0.0036144538316875696,
    -0.014621024020016193,
    0.027539826929569244,
    -0.018082672730088234,
    -0.03223911672830582,
    -0.011857330799102783,
    0.011000294238328934,
    0.021939130499958992,
    0.019541164860129356,
    -0.030450206249952316,
    0.01870325207710266,
    0.008215978741645813,
    0.010164094157516956,
    0.005839093122631311,
    -0.0007665493758395314,
    0.041285887360572815,
    -0.03187301754951477,
    0.012431291863322258,
    -0.02554955519735813,
    -0.01516569685190916,
    -0.020585186779499054,
    0.04627478867769241,
    0.011574918404221535,
    0.04289070516824722,
    -0.020552631467580795,
    -0.01600411906838417,
    -0.03550231084227562,
    0.006335296202450991,
    0.05937417224049568,
    0.03060079552233219,
    -0.015148223377764225,
    -0.027653632685542107,
    0.010515895672142506,
    -0.026747554540634155,
    -0.005388904828578234,
    -0.01721121184527874,
    0.010716249234974384,
    0.026079678907990456,
    0.020422428846359253,
    0.028367727994918823,
    0.01970166712999344,
    0.007845931686460972,
    -0.003653232241049409,
    -0.0014742885250598192,
    0.009662145748734474,
    -0.009876696392893791,
    -0.008697828277945518,
    0.020117487758398056,
    -0.01674829237163067,
    0.0052495235577225685,
    -0.03992583230137825,
    0.04623747617006302,
    -0.0006799210677854717,
    0.016743339598178864,
    -0.011798164807260036,
    0.06861621886491776,
    0.005792737938463688,
    0.020490601658821106,
    -0.005774150136858225,
    -0.060766614973545074,
    -0.0010443245992064476,
    0.026677237823605537,
    0.025530308485031128,
    0.009693755768239498,
    -0.017682382836937904,
    0.007530080620199442,
    -0.015690095722675323,
    -0.04360481724143028,
    -0.01453592162579298,
    0.007748787757009268,
    0.04437946528196335,
    0.009172722697257996,
    -0.03597608208656311,
    0.028515268117189407,
    0.0030131398234516382,
    -0.004791411105543375,
    -0.018249891698360443,
    -0.049296438694000244,
    -0.010252700187265873,
    -0.009501702152192593,
    -0.003106345422565937,
    0.08135411143302917,
    0.0006520499591715634,
    0.038398582488298416,
    0.011157149448990822,
    0.02076208032667637,
    0.057385820895433426,
    -0.004193044733256102,
    0.005013058893382549,
    -0.019408512860536575,
    0.017961429432034492,
    -0.025236790999770164,
    -0.05916766822338104,
    0.004323796834796667,
    0.005456916522234678,
    -0.034681178629398346,
    -0.005015108268707991,
    -0.02743205428123474,
    -0.02797197364270687,
    0.016495948657393456,
    -0.01961052231490612,
    -0.006390386261045933,
    0.05978405475616455,
    -0.005381511524319649,
    0.009398683905601501,
    -0.021517030894756317,
    -0.016899293288588524,
    0.00504653574898839,
    -0.004036975558847189,
    0.0012969870585948229,
    0.04502878338098526,
    -0.024686675518751144,
    0.007068033330142498,
    -0.02735724486410618,
    0.003393664723262191,
    0.000913545023649931,
    -0.005123266950249672,
    0.022424478083848953,
    0.0162159763276577,
    -0.018998706713318825,
    0.06408838927745819,
    -0.02899046801030636,
    0.01364470086991787,
    -0.019921252503991127,
    0.04903232678771019,
    -0.07874143868684769,
    -0.027630461379885674,
    0.02120542712509632,
    -0.007911070249974728,
    -0.03610140457749367,
    -0.025611508637666702,
    -0.06676161289215088,
    0.05163733661174774,
    -0.006887397263199091,
    -0.009330864995718002,
    -0.04501044750213623,
    -0.007429261226207018,
    -0.0011500868713483214,
    -0.016870718449354172,
    -0.009979606606066227,
    -0.01681671477854252,
    -0.04650338366627693,
    0.07667703181505203,
    0.0011810672003775835,
    0.012713226489722729,
    0.017119616270065308,
    0.04644770547747612,
    0.02134152315557003,
]

In [ ]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cosine_similarity(list1, list2):
    # Convert lists to vectors
    vector1 = np.array(list1)
    vector2 = np.array(list2)

    # Calculate cosine similarity
    similarity = dot(vector1, vector2) / (norm(vector1) * norm(vector2))
    return similarity


# Example usage:
similarity_score = cosine_similarity(ac4, ac2_headon)
print(f"Cosine Similarity: {similarity_score}")

In [ ]:
similarity_score = cosine_similarity(ac2_headon_2, ac2_headon)
print(f"Cosine Similarity: {similarity_score}")

In [ ]:
model = OllamaFunctions(model="phi3:latest", format="json", keep_alive=-1)

In [ ]:
model.invoke('hi there!, tell why 2+2 is 5?')

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.prompts import ChatPromptTemplate
import random
from langchain_core.tools import tool
from langchain_core.tools import tool
import random
from langchain_core.utils.function_calling import convert_to_openai_function


@tool
def fake_weather_api(city: str) -> str:
    """Check the weather in a specified city. The API is available randomly, approximately every third call."""

    if random.randint(1, 3) == 1:  # Approximately one-third chance
        return "Sunny, 22°C"
    else:
        return "Service temporarily unavailable"


tools = [fake_weather_api]
functions = [convert_to_openai_function(tool) for tool in tools]
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Make sure to use the tavily_search_results_json tool for information.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# Construct the Tools agent
agent = create_tool_calling_agent(model, functions, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "what is LangChain?"})

In [ ]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.tools import tool


@tool
def get_weather(location: str) -> str:
    """
    Get the weather for a location

    """
    return "weather in " + location + " is sunny with a high of 75 degrees"

@tool
def get_population(location: str) -> str:
    """
    Get the population of a location

    """
    return "the population of " + location + " is 100,000"


tools = [get_weather, get_population]
functions = [convert_to_openai_function(t) for t in tools]
tool_mapping = {"get_weather": get_weather, "get_population": get_population}

In [ ]:
functions

In [ ]:
model = model.bind_tools(functions)

In [ ]:
response = model.invoke("what is the weather in Boston and its population?")
response

In [ ]:
response.additional_kwargs["function_call"]

In [ ]:
response.additional_kwargs | itemgetter("arguments")

In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent